In [ ]:
# Salivary gland cancer
# CRTC1-MAML2 (translocation) [HSA:23373 84441] [KO:K15309 K06061]
# MYB-NF1B (translocation) [HSA:4602] [KO:K09420]
# ETV6-NTRK3 (translocation) [HSA:4916] [KO:K05101]
# EGFR (overexpression) [HSA:1956] [KO:K04361]
# HER2 (overexpression) [HSA:2064] [KO:K05083]
# CDKN2A (deletion) [HSA:1029] [KO:K06621]

In [1]:
protein2disease = {}
with open('Protein_Disease', 'r') as infile:
    all_lines = infile.readlines()

for line in all_lines:
    pid = line.rstrip().split(';')[0]
    disease = line.rstrip().split(';')[1:]
    protein2disease[pid] = disease

In [2]:
protein2disease['P24385']

['Multiple myeloma']

In [5]:
#DISEASE: Hodgkin lymphoma	
# Gene: P25963   	O00221   	Q04864  P25445
# Oral cancer
# 	P04637 	P42771  	P00533  	P01106  	P01111    	P01116 	P24385

# ids = ['P25963', 'O00221', 'Q04864', 'P25445', 'P04637', 'P42771', 'P00533', 'P01106', 'P01111', 'P01116',  'P24385']
ids = list(protein2disease.keys())
import requests, sys
import json
from tqdm import tqdm
ppi_go_terms = {}
remain_proteins = []
for uniprotid in tqdm(ids):
     
    requestURL = "https://www.ebi.ac.uk/QuickGO/services/annotation/search?geneProductId="+uniprotid
    r = requests.get(requestURL, headers={ "Accept" : "application/json"})
    if r.ok:
        responseBody = r.text
        data=json.loads(responseBody)
        for k in data['results']: 
            go_id = k['goId'] 
            if  uniprotid not in ppi_go_terms.keys():
                ppi_go_terms[uniprotid] = go_id
            else:
                ppi_go_terms[uniprotid] = ppi_go_terms[uniprotid] + ';' + go_id
        if len(data['results']) == 0:
            remain_proteins.append(uniprotid)
    else:
        remain_proteins.append(uniprotid)

100%|██████████| 20/20 [00:22<00:00,  1.14s/it]


In [6]:
#https://www.uniprot.org/uniprot/P25963.fasta

ppi_seqs_terms = {}
remain_proteins = []
for uniprotid in tqdm(ids): 
    requestURL = 'https://www.uniprot.org/uniprot/' +uniprotid+'.fasta'
    r = requests.get(requestURL, headers={ "Accept" : "application/json"})
    if r.ok:
        responseBody = r.text
        seq = ''
        for line in  responseBody.split('\n')[1:]:
            seq += line.rstrip()
        ppi_seqs_terms[uniprotid] = seq  
    else:
        remain_proteins.append(uniprotid)

100%|██████████| 20/20 [00:25<00:00,  1.26s/it]


In [7]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint

import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "1"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [8]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [9]:
import keras
feature_len = 768
max_go_len = 128
max_seq_len = 1000

prot2emb = {}
for key, value in ppi_go_terms.items():
    X_go1 =  np.zeros((1,768))
    allgos = value.split(';') 
    allgos = list(set(allgos))
    count = 0
    for  go in  allgos:
        feature = np.load('../ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
        if count + feature.shape[0] > max_go_len:
            break
        X_go1 = np.concatenate((X_go1,feature ))    
        count += feature.shape[0]
    prot2emb[key] =  X_go1[1:] 
protein2onehot = {}
for key, value in ppi_seqs_terms.items():
    protein2onehot[key] =  protein_one_hot(value, max_seq_len)
        


In [10]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [11]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, Flatten
from keras.utils import multi_gpu_model
from keras.layers.merge import concatenate
from keras_radam import RAdam
from keras_lookahead import Lookahead


def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(4)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y


def build_cnn_gru_model(input_x, con_filters, gru_units):
    x = inception_block(input_x,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(gru_units, return_sequences=True))(input_x)
    x_gru = Dropout(0.3)(x_gru)
     
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x_gru)
    x = Concatenate()([x_a, x_b, x_c, x_gru_a, x_gru_b,   x_gru_c])
    x = Dense(256,activation='relu')(x)
    return x



def build_model():
    con_filters = 256
    gru_units = 64
    left_input_go = Input(shape=(max_go_len,feature_len))
    right_input_go = Input(shape=(max_go_len,feature_len))
    
    
    left_input_seq = Input(shape=(max_seq_len,20))
    right_input_seq = Input(shape=(max_seq_len,20))
    
     
 
     
    left_x_go = build_cnn_gru_model(left_input_go, con_filters, gru_units)
    right_x_go = build_cnn_gru_model(right_input_go, con_filters,gru_units)
    
    left_x_seq = build_cnn_gru_model(left_input_seq, con_filters//4, gru_units)
    right_x_seq = build_cnn_gru_model(right_input_seq, con_filters//4, gru_units)
     
    
   
    x =   Concatenate()([left_x_go  , right_x_go, left_x_seq, right_x_seq])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
  
     
    x = Dense(1)(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([left_input_go, right_input_go, left_input_seq, right_input_seq], output)
#     model = multi_gpu_model(model, gpus=2)
    optimizer = Lookahead(RAdam())

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


model = build_model()
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 768)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 128, 768)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1000, 20)     0                                            
___________________________________________________________________________

In [12]:
save_model_name = '../SC_CV/sc_GoplusSeq0-0.hdf5'

In [13]:
model.load_weights(save_model_name)

In [14]:
X_go = np.zeros((len(ids), max_go_len, 768))
X_seq = np.zeros((len(ids), max_seq_len, 20))

for i, idx in enumerate(ids):
    feature = prot2emb[idx]
    X_go[i, :feature.shape[0]] = feature
    X_seq[i] = protein2onehot[idx]
    

In [15]:
sim_scores = np.zeros((len(ids), len(ids)))
for i in range(len(ids)):
    for j in range(len(ids)):
        if i != j:
            X_input = [X_go[i].reshape(1, max_go_len,768), X_go[j].reshape(1, max_go_len,768), 
                       X_seq[i].reshape(1, max_seq_len,20), X_seq[j].reshape(1, max_seq_len,20)]
            sim_scores[i,j] = model.predict(X_input)
             

In [16]:
sim_scores

array([[0.        , 0.99759406, 0.99512345, 0.94430417, 0.99969065,
        0.96155316, 0.9994753 , 0.99911433, 0.89677787, 0.88262373,
        0.99911875, 0.99678516, 0.99900299, 0.99985194, 0.99974495,
        0.99820638, 0.99730265, 0.9995684 , 0.99789619, 0.99856693],
       [0.9986676 , 0.        , 0.99138147, 0.99900442, 0.9996388 ,
        0.99180692, 0.99953675, 0.99420953, 0.99857426, 0.9988305 ,
        0.99756026, 0.99968398, 0.99551743, 0.99697936, 0.99946517,
        0.99373263, 0.99779153, 0.99947053, 0.99651104, 0.99248523],
       [0.99986994, 0.99197406, 0.        , 0.71268326, 0.99936181,
        0.66460705, 0.99897742, 0.99896514, 0.5645237 , 0.51606691,
        0.99889153, 0.98312277, 0.99877435, 0.9998914 , 0.99965405,
        0.99623066, 0.99500102, 0.99930286, 0.99640924, 0.99824727],
       [0.99540085, 0.99932575, 0.97840172, 0.        , 0.99975342,
        0.99480361, 0.99948704, 0.98036814, 0.99932349, 0.9996717 ,
        0.99635184, 0.99982291, 0.99100131, 0

In [ ]:
common_scores =  np.zeros((len(ids), len(ids)))
for i in range(len(ids)):
    for j in range(len(ids)):
        if i != j:
            idi = ids[i]
            idj = ids[j]
            common_scores = 